# Lab 4: Capstone - Build Your Own Agent

**Objective:** Apply everything you've learned to build a complete, functional AI agent.

**Duration:** ~45 minutes

**What You'll Do:**
- Choose an agent type and framework
- Design and implement your agent
- Test and refine your agent
- Prepare a brief demo

## Part 1: Choose Your Project

Select one of the following agent types (or propose your own!):

### Option A: Code Assistant Agent
An agent that helps developers with code-related tasks.

**Capabilities:**
- Analyze code for bugs or issues
- Explain what code does
- Suggest improvements
- Generate documentation

### Option B: Research Agent
An agent that conducts research on topics.

**Capabilities:**
- Search for information
- Synthesize findings from multiple sources
- Create structured summaries
- Save and organize notes

### Option C: Data Analyst Agent
An agent that helps analyze and understand data.

**Capabilities:**
- Load and process CSV/JSON data
- Calculate statistics
- Identify patterns and trends
- Create visualizations

### Option D: Your Own Idea!
Build an agent for a use case you care about.

In [ ]:
# Document your choice
project_choice = {
    "agent_type": "",  # A, B, C, or D
    "description": "",  # What does your agent do?
    "framework": "",  # LangChain, AutoGen, CrewAI, or LangGraph
    "why_this_framework": "",  # Why did you choose this framework?
    "tools_needed": [],  # What tools will your agent need?
    "success_criteria": ""  # How will you know it works?
}

print("Fill in your project choice above!")

## Part 2: Setup

Import the libraries you need based on your chosen framework.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found"
print("Environment configured!")

In [ ]:
# Import based on your framework choice

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
from langchain.memory import ConversationBufferMemory

# AutoGen imports (uncomment if using)
# from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# CrewAI imports (uncomment if using)
# from crewai import Agent, Task, Crew, Process

# LangGraph imports (uncomment if using)
# from typing import TypedDict, Annotated
# from langgraph.graph import StateGraph, END, START
# from langgraph.checkpoint.memory import MemorySaver
# import operator

print("Imports complete!")

## Part 3: Define Your Tools

Create the tools your agent needs.

In [ ]:
# Example tools - modify or replace based on your agent type

# Code Assistant Tools
@tool
def analyze_code(code: str) -> str:
    """Analyze code and identify potential issues.
    
    Args:
        code: The code to analyze
    """
    # TODO: Implement actual analysis
    return f"Analyzing code ({len(code)} characters)...\nAnalysis complete."

# Research Tools
@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
    """
    # TODO: Implement actual search (use Tavily if available)
    return f"Search results for '{query}': Found 5 relevant results."

@tool
def save_note(title: str, content: str) -> str:
    """Save a research note.
    
    Args:
        title: The note title
        content: The note content
    """
    # TODO: Implement actual storage
    return f"Note saved: {title}"

# Data Analyst Tools
@tool
def calculate_stats(data: str) -> str:
    """Calculate statistics on numerical data.
    
    Args:
        data: Comma-separated numbers
    """
    try:
        numbers = [float(x.strip()) for x in data.split(',')]
        avg = sum(numbers) / len(numbers)
        return f"Count: {len(numbers)}, Sum: {sum(numbers)}, Average: {avg:.2f}"
    except:
        return "Error: Could not parse data"

print("Sample tools defined!")
print("Modify or add tools based on your agent type.")

In [ ]:
# TODO: Define your custom tools here
# @tool
# def my_custom_tool(param: str) -> str:
#     """Description of what this tool does."""
#     return "result"

## Part 4: Build Your Agent

Implement your agent using your chosen framework.

In [ ]:
# LangChain Implementation Template

def build_langchain_agent():
    """Build a LangChain ReAct agent."""
    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Define tools
    tools = [analyze_code, search_web, save_note]  # Modify this list
    
    # Get prompt
    prompt = hub.pull("hwchase17/react")
    
    # Create agent
    agent = create_react_agent(llm, tools, prompt)
    
    # Create executor with memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
        max_iterations=10,
        handle_parsing_errors=True
    )
    
    return executor

# Uncomment if using LangChain:
# my_agent = build_langchain_agent()
# print("LangChain agent built!")

In [ ]:
# AutoGen Implementation Template

def build_autogen_team():
    """Build an AutoGen multi-agent team."""
    
    llm_config = {
        "config_list": [{
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY")
        }],
        "temperature": 0.7
    }
    
    # TODO: Define your agents
    # agent1 = AssistantAgent(
    #     name="Agent1",
    #     system_message="Your role...",
    #     llm_config=llm_config
    # )
    
    # TODO: Create user proxy and group chat
    
    pass

# Uncomment if using AutoGen:
# my_team = build_autogen_team()
# print("AutoGen team built!")

In [ ]:
# CrewAI Implementation Template

def build_crewai_crew():
    """Build a CrewAI crew."""
    
    # TODO: Define agents
    # agent1 = Agent(
    #     role="Role Name",
    #     goal="Agent's goal",
    #     backstory="Agent's backstory",
    #     verbose=True
    # )
    
    # TODO: Define tasks
    # task1 = Task(
    #     description="Task description",
    #     expected_output="What should be produced",
    #     agent=agent1
    # )
    
    # TODO: Create crew
    # crew = Crew(
    #     agents=[agent1],
    #     tasks=[task1],
    #     process=Process.sequential
    # )
    
    pass

# Uncomment if using CrewAI:
# my_crew = build_crewai_crew()
# print("CrewAI crew built!")

In [ ]:
# LangGraph Implementation Template

def build_langgraph_agent():
    """Build a LangGraph workflow agent."""
    
    from typing import TypedDict, Annotated
    from langgraph.graph import StateGraph, END, START
    import operator
    
    # TODO: Define state
    # class AgentState(TypedDict):
    #     messages: Annotated[list, operator.add]
    #     ...
    
    # TODO: Define nodes
    # def node1(state):
    #     return {...}
    
    # TODO: Build graph
    # graph = StateGraph(AgentState)
    # graph.add_node(...)
    # graph.add_edge(...)
    # app = graph.compile()
    
    pass

# Uncomment if using LangGraph:
# my_workflow = build_langgraph_agent()
# print("LangGraph workflow built!")

In [ ]:
# TODO: Build your agent using one of the templates above
# or create your own implementation

# my_agent = build_langchain_agent()
# OR
# my_team = build_autogen_team()
# OR
# my_crew = build_crewai_crew()
# OR
# my_workflow = build_langgraph_agent()

## Part 5: Test Your Agent

Run test cases to verify your agent works correctly.

In [ ]:
# Test cases for Code Assistant
code_assistant_tests = [
    "Analyze this code: def add(a, b): return a + b",
    "What does this function do: def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Suggest improvements for: for i in range(len(lst)): print(lst[i])"
]

# Test cases for Research Agent
research_agent_tests = [
    "Research the benefits of microservices architecture",
    "Find information about Python async programming",
    "What are the latest trends in AI development?"
]

# Test cases for Data Analyst
data_analyst_tests = [
    "Calculate statistics for: 10, 20, 30, 40, 50",
    "Analyze the trend in: 100, 120, 115, 130, 145, 160",
    "Find the average of: 85, 90, 78, 92, 88"
]

print("Test cases ready!")

In [ ]:
# TODO: Run your tests
# Choose the appropriate test cases for your agent type

# Example for LangChain agent:
# for test in code_assistant_tests:
#     print(f"\n{'='*50}")
#     print(f"Test: {test}")
#     result = my_agent.invoke({"input": test})
#     print(f"Result: {result['output']}")

In [ ]:
# Document your test results
test_results = {
    "test_1": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    },
    "test_2": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    },
    "test_3": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    }
}

print("Document your test results above!")

## Part 6: Prepare Your Demo

Create a demo-ready showcase of your agent.

In [ ]:
# Demo script
def run_demo():
    """Run a demo of your agent."""
    print("="*60)
    print("AGENT DEMO")
    print("="*60)
    
    # TODO: Add your demo steps
    print("\n1. Agent Introduction")
    print("   This agent does: [describe your agent]")
    
    print("\n2. Demo Task 1")
    # result1 = my_agent.invoke({"input": "..."})
    # print(f"   Result: {result1['output']}")
    
    print("\n3. Demo Task 2")
    # result2 = my_agent.invoke({"input": "..."})
    # print(f"   Result: {result2['output']}")
    
    print("\n" + "="*60)
    print("DEMO COMPLETE")
    print("="*60)

# Uncomment to run your demo:
# run_demo()

## Part 7: Reflection and Documentation

In [ ]:
# Final project documentation
project_summary = {
    "agent_name": "",
    "description": "",
    "framework_used": "",
    "tools_implemented": [],
    "key_features": [],
    "challenges_faced": "",
    "lessons_learned": "",
    "future_improvements": ""
}

print("Fill in your project summary!")

In [ ]:
# Framework comparison reflection
framework_reflection = {
    "framework_used": "",
    "why_good_fit": "",
    "limitations_encountered": "",
    "would_use_different_framework_for": ""
}

print("Reflect on your framework choice!")

## Capstone Checklist

Before presenting, verify:

- [ ] Agent runs without errors
- [ ] At least 2-3 tools are implemented
- [ ] Agent accomplishes its stated goal
- [ ] Demo can be completed in 2-3 minutes
- [ ] Code is reasonably organized
- [ ] You can explain how it works

## Workshop Summary

Congratulations on completing the Crafting Custom Agents Workshop!

### What You Learned

**Session 1:**
- ReAct pattern and agent fundamentals
- LangChain agents with custom tools
- AutoGen conversational multi-agent systems
- CrewAI role-based crews

**Session 2:**
- LangGraph stateful workflows
- Conditional routing and cycles
- Human-in-the-loop patterns
- Framework selection and trade-offs

### Framework Quick Reference

| Framework | Best For |
|-----------|----------|
| LangChain | Single agents with tools |
| AutoGen | Conversational multi-agent |
| CrewAI | Role-based pipelines |
| LangGraph | Complex stateful workflows |

### Keep Learning

- [LangChain Documentation](https://python.langchain.com/docs/)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [AutoGen Documentation](https://microsoft.github.io/autogen/)
- [CrewAI Documentation](https://docs.crewai.com/)

### Thank You!

We hope this workshop has given you the skills to build powerful AI agents for your projects!

In [ ]:
print("Workshop Complete!")
print("\nYou are now equipped to build AI agents using multiple frameworks.")
print("Go forth and build amazing things!")